## Import Libraries

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib nbagg
from matplotlib import animation, rc, cm
import IPython, io, urllib
rc('animation', html='html5')
from midiutil.MidiFile import MIDIFile
import scipy.io.wavfile
import scipy.signal as signal

## A generic class to simulate LLCAs

In [2]:
class LLCA:
    """
    A Life Like Cellular Automaton (LLCA)

    Inputs:
    * C: a binary matrix representing the cells where 1 stands for alive and 0 for dead.
    * rule: the rule game of the in the format 'BXSY' where X and Y are the birth and survival conditions.
            Example: GOL rule is "B3S23".
    """
    def __init__(self, C = np.random.rand(50, 50), rule = "B3S23"):
        self.C = np.array(C).astype(bool)
        self.rule = rule

    def parse_rule(self):
        """
        Parses the rule string
        """
        r = self.rule.upper().split("S")
        B = np.array([int(i) for i in r[0][1:] ]).astype(np.int64)
        S = np.array([int(i) for i in r[1] ]).astype(np.int64)
        return B, S

    def neighbors(self):
        """
        Returns the number of living neigbors of each cell.
        """
        C = self.C
        N = np.zeros(C.shape, dtype = np.int8) # Neighbors matrix
        N[ :-1,  :  ]  += C[1:  , :  ] # Living cells south
        N[ :  ,  :-1]  += C[ :  ,1:  ] # Living cells east
        N[1:  ,  :  ]  += C[ :-1, :  ] # Living cells north
        N[ :  , 1:  ]  += C[ :  , :-1] # Living cells west
        N[ :-1,  :-1]  += C[1:  ,1:  ] # Living cells south east
        N[1:  ,  :-1]  += C[ :-1,1:  ] # Living cells north east
        N[1:  , 1:  ]  += C[ :-1, :-1] # Living cells north west
        N[ :-1, 1:  ]  += C[1:  , :-1] # Living cells south west
        return N

    def iterate(self):
        """
        Iterates one time.
        """
        B, S = self.parse_rule()
        N = self.neighbors()
        C = self.C
        C1 = np.zeros(C.shape, dtype = np.int8)
        for b in B: C1 += ((C == False) & (N == b))
        for s in S: C1 += (C & (N == s))
        self.C[:] = C1 > 0

## The orginal Game of Life (rule B3S23)

In [3]:
# INITIAL CONFIGURATION

# grid size
N = 23
# number of frames (generations)
frameNumb = 125
# speed of interpolation (ms)
frameSpeed = 3000

t = np.linspace(0., 1., N+1)
X, Y = np.meshgrid(t, t)
f = 1
C0 = np.sin(2. * np.pi * f * X ) * np.sin(2. * np.pi * 2 * f * Y ) > -.1

# put new birth/survival conditions in here for new simulations
g = LLCA(C0, rule = "B3S23")

trueCount = []

# ANIMATION
def updatefig(*args):
    g.iterate()
    im.set_array(g.C)
    
    # loop through 2D matrix 1 row at a time
    for x in g.C:
        
        # count number of 'true' elements in each row and store index position in a new array
        where = np.where(x)
        
        # append the where array to new array
        trueCount.append(where)
        #print(where)
    
    #print(len(trueCount)) # length should be equal to the number of frames + 1!

    return im,

fig, ax = plt.subplots()
ax.axis('off')
im = plt.imshow(g.C, interpolation = "nearest", cmap = cm.cubehelix, animated=True)
anim = animation.FuncAnimation(fig, updatefig, frames=frameNumb, interval=frameSpeed, blit=True)

plt.close()
anim
#plt.show()

<IPython.core.display.Javascript object>

In [21]:
# Generate audio from 'true' cells in matrix each row to create a beat sequencer
# Copy all of this code and paste it below other rule sets to write MIDI files

fs = 44100     # in samples/second
duration = (frameSpeed/(N+1))/1000 # calculated from frameSpeed and grid size (N) above
#print(duration)

# Convert MIDI note number m into a frequency in Hz.
def mtof(m):
    return 440*np.power(2.0,(m-69)/12)

def generatesinewave(m,duration):
    t = np.linspace(0, duration, int(duration*fs))
    
    # Derive the ramp signal that goes up and then down again
    numSamplesGoingUp = int(duration*fs/2)
    numSamplesGoingDown = np.size(t) - numSamplesGoingUp
    envelopingFunction = np.concatenate((np.linspace(0, 1, numSamplesGoingUp), np.linspace(1, 0, numSamplesGoingDown)))
    
    frequency = mtof(m)
    mysinewave = np.sin(2*np.pi*t*frequency)# * envelopingFunction

    return mysinewave

# begin ---

#print(len(trueCount)) # should be equal to number of frames * grid size (N)
#print(trueCount[2]) # print a single index

# create your MIDI object
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

time = 0    # start at the beginning
tempo = (60000/(duration*1000)) # calculated from duration above to get bpm
print(duration)
print(tempo)
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, tempo)

# info for ALL MIDI
channel = 0
volume = 100
length = 0.125

transpose = 72 # transpose all MIDI values by this value
    
for arr in trueCount:  # for each array in trueCount
    
    for x in arr:  # for each row in array
        
        #print(arr)
        #print(len(x))
        
        if len(x) > 0: # if the array has more than 0 'live' cells... :)
            
            #print(x)
            #print(len(x))
            
            # add some MIDI notes
            channel = 0
            volume = 100
            time = time + 1 # start on beat 1 and increase
        
            for count,ele in enumerate(x):  # for each index in the array, do...
                
                #print(count) # prints all indexes individually
                #print(ele) # prints all elements individually
                
                if ele == 0:
                    pitch = 0+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 1:
                    pitch = 2+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 2:    
                    pitch = 4+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 3:    
                    pitch = 6+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 4:    
                    pitch = 7+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 5:    
                    pitch = 9+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 6:    
                    pitch = 11+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 7:    
                    pitch = 12+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 8:    
                    frequency8 = mtof(0+transpose)
                    pitch = 14+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 9:    
                    pitch = 16+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 10:
                    pitch = 18+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 11:
                    pitch = 19+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 12:    
                    pitch = 21+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 13:    
                    pitch = 23+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 14:    
                    pitch = 24+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 15:    
                    pitch = 26+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 16:    
                    pitch = 28+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 17:    
                    pitch = 30+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 18:    
                    pitch = 31+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 19:    
                    pitch = 33+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 20:
                    pitch = 35+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 21:
                    pitch = 36+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 22:    
                    pitch = 38+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
                elif ele == 23:    
                    pitch = 40+transpose  # pitch
                    mf.addNote(track, channel, pitch, time, length, volume)
            
        else:  # if the array has no 'live' cells... :(
            
            # add some MIDI notes
            channel = 0
            volume = 100
            time = time + 1 # start on beat 1 and increase
            pitch = 0  # pitch
            mf.addNote(track, channel, pitch, time, length, 0)

# write it to disk
with open("GOLMIDI.mid", 'wb') as outf:
    mf.writeFile(outf)

0.125
480.0


## Alternative rule: Day and Night (B3678S34678)

In [97]:
N = 100
t = np.linspace(0., 1., N+1)
X, Y = np.meshgrid(t, t)
f = 10
C0 = np.sin(2. * np.pi * f * X ) * np.sin(2. * np.pi * 2 * f * Y )  > 0.

g = LLCA(C0, rule = "B3678S34678")

def updatefig(*args):
    g.iterate()
    im.set_array(g.C)
    return im,


fig, ax = plt.subplots()
ax.axis('off')
im = plt.imshow(g.C, interpolation = "nearest", cmap = cm.Accent, animated=True)
anim = animation.FuncAnimation(fig, updatefig, frames=150, interval=50, blit=True)
plt.close()
anim
#plt.show()

<ipython-input-87-da59444a2c42>:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.C = np.array(C).astype(np.bool)


<IPython.core.display.Javascript object>

## Alternative rule: fractal-like B1S123

In [98]:
N = 200
C0 = np.zeros((N, N))
C0[1,1] = 1

# alternate taken from original GOL
#C0 = np.sin(2. * np.pi * f * X ) * np.sin(2. * np.pi * 2 * f * Y )  > -.1

g = LLCA(C0, rule = "B1S123")

def updatefig(*args):
    g.iterate()
    im.set_array(g.C)
    return im,


fig, ax = plt.subplots()
ax.axis('off')
im = plt.imshow(g.C, interpolation = "nearest", cmap = cm.binary, animated=True)
anim = animation.FuncAnimation(fig, updatefig, frames=150, interval=50, blit=True)
plt.close()
anim
#plt.show()

<ipython-input-87-da59444a2c42>:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.C = np.array(C).astype(np.bool)


<IPython.core.display.Javascript object>